# Seq2Seq

あるシーケンスから別のシーケンスへの変換を行うSeq2Seqというモデルを学び、機械翻訳へ応用する。

In [1]:
import os; os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings; warnings.filterwarnings("ignore")
import math
import random

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(
    width=20,
    with_test=True,
    label="ppl train",
    round=2,
    agg_fn=lambda s, w: math.exp(s / w)
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')


---

## 条件付き言語モデル

言語モデルに文脈以外の条件を付与する。

### 言語モデルへの条件付け

これまでの言語モデルは文脈を条件とした確率モデルであった。

$$
p(w_t|w_{<t})
$$

ここで、文脈以外の条件を追加してみる。

$$
p(w_t|w_{<t}, c)
$$

これは、なんらかの条件$c$に基づいた文章を生成するモデルと見られる。

例えば条件を画像とする場合、画像に基づいた文章を生成するモデルとなり、画像のキャプション生成などに使える。また条件を音声とする場合、音声に基づいた文章を生成するモデルとなり、音声認識などに使える。

では、文章を条件とする場合を考えてみる。この場合、文章から文章を生成するモデルとなる。これはどんなことに使えるだろう。

例えば文章要約が挙げられる。条件としてある文章を与え、そこから要点のみをまとめた新たな文章を生成する。また、機械翻訳も考えられそう。入力された文章から、同じ意味を持った別の言語の文章を生成する。

### RNNへの条件付け

RNNがこれらの条件を考慮するためにはどうすればよいだろうか。

といっても、条件から適当に特徴量を抽出し、それをRNNのどこかに繋げるだけでよい。適当なところで、足したり、結合して線形変換したり、やりようはいくらでもある。また、隠れ状態の初期値として条件を与える方法も考えられる。これまで0ベクトルとしていたところに、条件から抽出した特徴量を与える。

ここで、特徴抽出モデルをNNとすると、当然そのNNまで勾配が届くので、RNNと同時に学習することができる。実際に、CNNとRNNを繋げた画像のキャプション生成モデルが提案されている[1]。CNNで抽出した画像特徴量をRNNに隠れ状態の初期値として与えている。

[1] [Show and Tell: A Neural Image Caption Generator](https://arxiv.org/abs/1411.4555)


---

## Seq2Seq

*Sequence to Sequence*。*Encoder-Decoder Model*とも。

これまでRNNを言語モデルとして使ってきたが、RNNにはもう少しできることがある。それは特徴抽出である。ある時系列データを入力したときに得られる最後の隠れ状態には全ての時刻の情報が含まれており、これは特徴量と見ることが出来る。

ここで、時系列データを言語モデルへの条件として扱うことを考える。前節より、言語モデルをRNNとすると、隠れ状態の初期値として条件を与えることができる。そして時系列データの特徴抽出にはRNNが使えるため、最終的に2つのRNNを繋げたモデルができる。このモデルは時系列データ（Sequence）から時系列データへの変換を行うモデルと見られ、**Sequence to Sequence**または**Seq2Seq**と呼ばれる。

Seq2Seqは時系列データからの特徴抽出を行うRNNと時系列データを生成するRNNに分かれている。前者を**Encoder**、後者を**Decoder**と呼ぶ。ここから、Seq2Seqは**Encoder-Decoderモデル**とも呼ばれる。

Seq2Seqを用いることで時系列データから時系列データの生成が可能になる。言語モデルと関連した例を挙げると、機械翻訳や文章要約などがある。

本章では機械翻訳でSeq2Seqを学ぶ。


---

## 教師データの作成

翻訳モデルの学習に必要な教師データを作成する。

### 対訳コーパス

翻訳モデルを作るには、同じ意味を持つ文章が複数の言語でまとまっているデータが必要。このようなデータは対訳コーパスと呼んだりする。

本章では以下のデータセットから日本語と英語の対訳コーパスを使用する。

- [iwslt2017  |  TensorFlow Datasets](https://www.tensorflow.org/datasets/community_catalog/huggingface/iwslt2017?hl=ja#iwslt2017-en-ja)

In [ ]:
ds = tfds.load(
    "huggingface:iwslt2017/iwslt2017-en-ja",
    data_dir="data",
    split="train"
)
ds = list(ds.as_numpy_iterator())

In [4]:
data_ja = []
data_en = []
for sample in ds:
    ja = sample["translation"]["ja"].decode()
    en = sample["translation"]["en"].decode()
    data_ja.append(ja)
    data_en.append(en)

print("num of data:", len(data_ja), "\n")
for _ in range(5):
    i = random.randint(0, len(data_ja))
    print(data_ja[i])
    print(data_en[i])
    print()

num of data: 223108 

腐敗が 病的に民主主義を 破壊してしまうということです つまり どのようなシステムであっても その構成員がごく少数の 構成員によって選ばれる場合には その構成員がごく少数の 構成員によって選ばれる場合には それは ごく少数の構成員が ごくごく少数の構成員が 改革を阻止できることを意味します
It's a pathological, democracy-destroying corruption, because in any system where the members are dependent upon the tiniest fraction of us for their election, that means the tiniest number of us, the tiniest, tiniest number of us, can block reform.

これは大変なことです
This is a big deal.

疑問というのは 西洋社会におけるすべての議論は 課税レベルに関することです
We ask the question -- the whole debate in the Western world is about the level of taxation.

つまり大きな意味では 技術というのは何も こんな機器だけをさすのではないのです 習慣、テクニック、心理的手法 なども含めて 技術と呼べるのです
So in a broad sense, we don't need to think about technology as only little gadgets, like these things here, but even institutions and techniques, psychological methods and so forth.

ひも理論が難しいなんて人もいますが 楽なもんです
And some people think that string theory is tough.



書き出し

In [5]:
textfile_ja = "data/iwslt2017_ja.txt"
with open(textfile_ja, "w") as f:
    f.write("\n".join(data_ja))

textfile_en = "data/iwslt2017_en.txt"
with open(textfile_en, "w") as f:
    f.write("\n".join(data_en))

読み込み

In [3]:
textfile_ja = f"data/iwslt2017_ja.txt"
textfile_en = f"data/iwslt2017_en.txt"

with open(textfile_ja) as f:
    data_ja = f.read().splitlines()
with open(textfile_en) as f:
    data_en = f.read().splitlines()

#### 前処理

日本語、英語別々にトークナイザを作成する。

In [4]:
tokenizer_prefix_ja = f"models/tokenizer_iwslt2017_ja"
tokenizer_prefix_en = f"models/tokenizer_iwslt2017_en"
pad_id = 3
vocab_size = 8000

In [ ]:
spm.SentencePieceTrainer.Train(
    input=textfile_ja,
    model_prefix=tokenizer_prefix_ja,
    vocab_size=vocab_size,
    pad_id=pad_id
)

spm.SentencePieceTrainer.Train(
    input=textfile_en,
    model_prefix=tokenizer_prefix_en,
    vocab_size=vocab_size,
    pad_id=pad_id
)

In [5]:
sp_ja = spm.SentencePieceProcessor(f"{tokenizer_prefix_ja}.model")
sp_en = spm.SentencePieceProcessor(f"{tokenizer_prefix_en}.model")

unk_id = sp_ja.unk_id()
bos_id = sp_ja.bos_id()
eos_id = sp_ja.eos_id()
pad_id = sp_ja.pad_id()

n_vocab_ja = len(sp_ja)
n_vocab_en = len(sp_en)
print("num of vocabrary (ja):", n_vocab_ja)
print("num of vocabrary (en):", n_vocab_en)

num of vocabrary (ja): 8000
num of vocabrary (en): 8000


トークン化

In [6]:
data_ids_ja = sp_ja.encode(data_ja)
data_ids_en = sp_en.encode(data_en)

BOS, EOSの追加

In [7]:
for ids_ja, ids_en in zip(data_ids_ja, data_ids_en):
    ids_en.insert(0, bos_id)
    ids_ja.append(eos_id)
    ids_en.append(eos_id)

### 教師データの作成

入力文と正解のペアを作成する。

通常のRNNLMでは、トークン列とそれを1つずらしたトークン列がペアとなる。Seq2SeqではこのペアがDecoderへの入力と正解となり、これに加えてEncoderへの入力を用意する。

例）

入力（Encoder） | 入力（Decoder） | 正解
--- | --- | ---
夏 休み が 終わり ました 。 \<EOS> | \<BOS> Summer vacation is over . | Summer vacation is over . \<EOS>
ツイッター は 亡くなり ました 。 \<EOS> | \<BOS> Twitter is dead . | Twitter is dead . \<EOS>
今日 から X で 暮らし ましょう 。 \<EOS> | \<BOS> Let 's live in X from today . | Let 's live in X from today . \<EOS>

`DataLoader`の作成。

In [8]:
class TextDataset(Dataset):
    def __init__(self, data_ids_ja, data_ids_en):
        self.data_ja = [torch.tensor(ids) for ids in data_ids_ja]
        self.data_en = [torch.tensor(ids) for ids in data_ids_en]
        self.n_data = len(self.data_ja)

    def __getitem__(self, idx):
        ja = self.data_ja[idx]
        en = self.data_en[idx]
        x_enc = ja # Encoderへの入力
        x_dec = en[:-1] # Decoderへの入力
        y_dec = en[1:] # Decoderの出力
        return x_enc, x_dec, y_dec

    def __len__(self):
        return self.n_data

def collate_fn(batch): # padding
    x_enc, x_dec, y_dec= zip(*batch)
    x_enc = pad_sequence(x_enc, batch_first=True, padding_value=pad_id)
    x_dec = pad_sequence(x_dec, batch_first=True, padding_value=pad_id)
    y_dec = pad_sequence(y_dec, batch_first=True, padding_value=pad_id)
    return x_enc, x_dec, y_dec

dataset = TextDataset(data_ids_ja, data_ids_en)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])
print("num of train data:", len(train_dataset))
print("num of test data:", len(test_dataset))

batch_size = 32
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    drop_last=True,
    collate_fn=collate_fn
)

# example
x_enc, x_dec, y_dec = next(iter(train_loader))
x_enc.shape, x_dec.shape, y_dec.shape

num of train data: 178487
num of test data: 44621


(torch.Size([32, 72]), torch.Size([32, 67]), torch.Size([32, 67]))


---

## モデル構築

Encoder、Decoderを作成し、Seq2Seqモデルを作成する。

### Encoder

入力文を入れて固定長のベクトルを出力するだけのRNN。

ここで、何をDecoderに渡すかを考える。先で「最後の隠れ状態」と言ったが、実はそれ以外にもいくつか選択肢がある。

Decoderに渡したい隠れ状態として満たしてほしい条件は以下である。

- （padを除いた）全ての入力を参照して出力されている
- 固定長

これらを全て満たしていれば何でもよい。

まず考えられるのは最後の隠れ状態。これが一番シンプルで実装も簡単。

ただこれだと学習時にpadトークンが隠れ状態に関与してしまう。多くの場合、推論時にpadトークンは存在しないため、学習時もpadトークンを考慮しない出力をさせた方が良さそう。また、padの数が多くなるにつれて隠れ状態がある一定の値に収束してしまう（経験談）。RNNに同じトークンを何度も入力することで隠れ状態が収束してしまうみたい。そうなってしまうと、入力文に依る隠れ状態の違いが少なくなり、入力文を考慮した出力が行えない。

ではどうするかというと、「padを除いた最後の隠れ状態」とする。eosトークンが入力された時点の隠れ状態とも言える。これで上記の問題を解決できる。

他にもいくつかの案が考えられる。例えば、「padを除いた全ての隠れ状態の平均」とか。これも条件を満たす。ただ今回は無難に、「padを除いた最後の隠れ状態」とする。

ではEncoderを実装する。これまでのRNN同様、適当にLSTMと線形層で作る。ここで精度向上のため、以下の工夫を加える。

- LSTM層を3層に増やす
- 残差結合を取り入れる

これらの工夫はSeq2Seq固有のものではなく、一般的なRNNに応用できる。例えば前章までのRNNLMにも適用でき、精度向上に繋がるはず。

In [9]:
class Encoder(nn.Module):
    def __init__(
        self,
        n_vocab,
        embed_size,
        hidden_size,
        dropout=0.2,
    ):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm1 = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, hidden_size)
        self.fc_skip = nn.Linear(embed_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        x: (batch_size, seq_len)
        """
        eos_pos = x == eos_id
            # eosに対応する位置のみがTrueとなったTensor: (batch_size, seq_len)
        x = self.embedding(x) # (batch_size, seq_len, embed_size)

        skip = self.fc_skip(x)
        hs1, _ = self.lstm1(x) # (batch_size, seq_len, hidden_size)
        hs1 = hs1 + skip
        hs1 = self.dropout(hs1)

        skip = hs1
        hs2, _ = self.lstm2(hs1)
        hs2 = hs2 + skip
        hs2 = self.dropout(hs2)

        skip = hs2
        hs3, _ = self.lstm3(hs2)
        hs3 = hs3 + skip
        hs3 = self.dropout(hs3)

        h1 = hs1[eos_pos] # (batch_size, hidden_size)
        h2 = hs2[eos_pos]
        h3 = hs3[eos_pos]
        return h1, h2, h3

全ての層からの隠れ状態をタプルで出力する。もちろん、最後の層だけ使うみたいな事もできる。

### Decoder

Encoderから出力された隠れ状態を受け取り、出力文を生成するRNN。Encoder同様、LSTMと線形層にいくつかの工夫を加えて作る。LSTM層の数はEncoderと同じにする。

In [10]:
class Decoder(nn.Module):
    def __init__(
        self,
        n_vocab,
        embed_size,
        hidden_size,
        dropout=0.2,
    ):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_size)
        self.lstm1 = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.lstm3 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, n_vocab)
        self.fc_skip = nn.Linear(embed_size, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, hc):
        hc1, hc2, hc3 = hc
        x = self.embedding(x) # (batch_size, seq_len, embed_size)

        skip = self.fc_skip(x) # (batch_size, seq_len, hidden_size)
        hs, hc1 = self.lstm1(x, hc1) # (batch_size, seq_len, hidden_size)
        hs = hs + skip
        hs = self.dropout(hs)

        skip = hs
        hs, hc2 = self.lstm2(hs, hc2)
        hs = hs + skip
        hs = self.dropout(hs)

        skip = hs
        hs, _ = self.lstm3(hs)
        hs = hs + skip
        hs = self.dropout(hs)

        y = self.fc(hs) # (batch_size, seq_len, n_vocab)
        return y, (hc1, hc2, hc3)

`hc`には各LSTM層に対応する隠れ状態がタプルで含まれていることを想定している。具体的に何が入るかは次へ。

### Seq2Seq

EncoderとDecoderを合わせて、入力から出力までの一連の処理を行うモデルを作る。

In [11]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, x_enc, x_dec):
        h = self.encoder(x_enc)
        hc = self.get_hc(h)
        y, _ = self.decoder(x_dec, hc)
        return y

    def get_hc(self, h):
        h = torch.stack(h, dim=0).unsqueeze(1)
        c = torch.zeros_like(h)
        hc = zip(h, c)
        return hc

Encoderから受け取った`h`（`(h1, h2, h3)`）から`hc`（`(hc1, hc2, hc3)`）を作成し、Decoderに渡す。なお`hcn`は`(hn, cn)`で`cn`は0ベクトル。

In [12]:
hidden_size, embed_size = 512, 512
encoder = Encoder(n_vocab_ja, embed_size, hidden_size)
decoder = Decoder(n_vocab_en, embed_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 25,691,456



---

## 実践

実際にSeq2Seqを学習させて翻訳を行ってみる。

### 学習

In [13]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)
def loss_fn(y, t):
    loss = cross_entropy(y.reshape(-1, n_vocab_ja), t.ravel())
    return loss

@torch.no_grad()
def eval_model(model):
    model.eval()
    losses = []
    for x_enc, x_dec, y_dec in test_loader:
        x_enc = x_enc.to(device)
        x_dec = x_dec.to(device)
        y_dec = y_dec.to(device)

        y = model(x_enc, x_dec)
        loss = loss_fn(y, y_dec)
        losses.append(loss.item())
    loss = sum(losses) / len(losses)
    ppl = math.exp(loss)
    return ppl

def train(model, optimizer, n_epochs, prog_unit=1):
    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x_enc, x_dec, y_dec in train_loader:
            optimizer.zero_grad()
            x_enc = x_enc.to(device)
            x_dec = x_dec.to(device)
            y_dec = y_dec.to(device)

            y = model(x_enc, x_dec)
            loss = loss_fn(y, y_dec)
            loss.backward()
            optimizer.step()
            prog.update(loss.item())

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.2f}", no_step=True)
        prog.memo()

In [14]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [15]:
train(model, optimizer, n_epochs=20, prog_unit=1)

 1/20:                        0% [00:00:00.98] ppl train: 7654.46 

 1/20: #################### 100% [00:04:53.78] ppl train: 149.26, test: 90.66 
 2/20: #################### 100% [00:04:46.93] ppl train: 85.17, test: 70.28 
 3/20: #################### 100% [00:04:47.46] ppl train: 69.05, test: 61.11 
 4/20: #################### 100% [00:04:47.45] ppl train: 60.01, test: 55.41 
 5/20: #################### 100% [00:04:47.37] ppl train: 53.89, test: 51.44 
 6/20: #################### 100% [00:04:47.91] ppl train: 49.29, test: 48.57 
 7/20: #################### 100% [00:04:47.46] ppl train: 45.79, test: 46.47 
 8/20: #################### 100% [00:04:47.44] ppl train: 42.86, test: 45.14 
 9/20: #################### 100% [00:04:47.50] ppl train: 40.44, test: 43.57 
10/20: #################### 100% [00:04:49.70] ppl train: 38.41, test: 42.47 
11/20: #################### 100% [00:04:48.26] ppl train: 36.57, test: 41.65 
12/20: #################### 100% [00:04:47.66] ppl train: 34.96, test: 40.98 
13/20: #################### 100% [00:04:45.36] ppl train: 33.53

In [16]:
model_path = "models/lm_seq2seq.pth"
torch.save(model.state_dict(), model_path)

### 翻訳

作成したモデルに日本語文を入力し、英語に翻訳して出力する。決定的な出力にする。

In [17]:
def token_sampling(y, decisive=True):
    y = y.squeeze(0, 1)
    if decisive:
        token = y.argmax().item()
    else:
        y[unk_id] = -torch.inf
        probs = F.softmax(y, dim=-1)
        token, = random.choices(range(n_vocab_en), weights=probs)
    return token


bos_id = sp_en.bos_id()
eos_id = sp_en.eos_id()
@torch.no_grad()
def translate(
    model: nn.Module,
    in_text: str, # 入力文（日本語）
    max_len: int = 100, # 出力のトークン数の上限
    decisive: bool = True, # サンプリングを決定的にするか
) -> str:
    model.eval()
    in_ids = sp_ja.encode(in_text)
    in_ids = torch.tensor(in_ids + [eos_id], device=device).unsqueeze(0)

    h_enc = model.encoder(in_ids)
    hc = model.get_hc(h_enc)
    next_token = bos_id

    token_ids = []
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device).reshape(1, 1)
        y, hc = model.decoder(x, hc)
        next_token = token_sampling(y, decisive)
        token_ids.append(next_token)

    sentence = sp_en.decode(token_ids)
    return sentence

まずは訓練データから。

In [19]:
n = 5
for _ in range(n):
    i = random.randint(0, len(train_dataset))
    x, _, t = train_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: 水面すれすれで 1つを除いてすべて無人島です そこには約35人の管理者が住んでいます
output: It's a lot of rainwater, but they're also heavily hotterday than the Gulf Coast Redwood rainforest.
answer: They're very low to the water, and they're all uninhabited, except one island has about 35 caretakers on it.

input: このような文化が― 実際には残りすべての世界は実際― 西側諸国と比べて とても弱い立場にありましたが こういった国は西側諸国を 理解するように強いられてきました なぜなら 社会の中に西洋の存在があったからです
output: And the culture of violence is very similar to the extent that we have been able to cope with the Arab Springs, and then finally, in spite of the poorest congressionalities, the oldest Arab Springspherence, but also the same thing as the Arab Spring.
answer: Whereas those cultures -- virtually the rest of the world, in fact, which have been in a far weaker position, vis-a-vis the West -- have been thereby forced to understand the West, because of the West's presence in those societies.

input: あなたはあなたを構成する部品を 合体させたものなのです
output: You can't afford to use your genomes.
answer: You are the sum of your

あまりよくないね。

訓練データに含まれていないものも試してみる。

In [20]:
# test data
for _ in range(n):
    i = random.randint(0, len(test_dataset))
    x, _, t = test_dataset[i]
    x = sp_ja.decode(x.tolist())
    t = sp_en.decode(t.tolist())
    print("input:", x)
    print("output:", translate(model, x))
    print("answer:", t)
    print()

input: 素晴らしく 満ち足りた気持ちになります
output: It was a lot of patience.
answer: I get this extraordinary feeling of well-being.

input: 嫁ぎ先で彼女はゾッとしました トイレが無いのです
output: And she's getting rid of the pesticides.
answer: And she was horrified to get there and find that they didn't have a toilet.

input: 数ヶ月前ニューヨークのタイムズスクウェアで パキスタン系イスラム教徒の男が車の爆破を企てました
output: In 2005, I was invited to write a book called "The Seventy Jobs" was published in 1998.
answer: Like a couple months ago in Times Square in New York, there was this Pakistani Muslim guy who tried to blow up a car bomb.

input: 私はそれをふるさとの安全と呼んでいます
output: And I'm trying to protect them.
answer: And I call it "hometown security."

input: じっと見ていると画面から客席の方に浮かびあがってきます
output: You can see the imagery from the beginning, and they're going to be able to read.
answer: And if you sit here long enough, it'll float off the page into the audience.



In [21]:
# original
sentences = [
    "ありがとう",
    "猫はかわいいのです",
    "上手く文章が書けるようになりました"
]

for sentence in sentences:
    print("input:", sentence)
    print("output:", translate(model, sentence))
    print()

input: ありがとう
output: Thank you.

input: 猫はかわいいのです
output: You know, they're nuts.

input: 上手く文章が書けるようになりました
output: They were playing video games.



びみょう。